# Tools

In [1]:
import pandas as pd
from typing import Dict, Any
from app.core import config
from datetime import datetime
import re
import httpx
import unicodedata
from thefuzz import fuzz
from IPython.display import display, HTML
from app.utils.functions import (
    get_mapping, 
    create_simplified_hierarchy, 
    execute_sp, 
    extract_all_descendants_for_list, 
    get_children_by_label, 
    preprocessing_data,
    get_col_info, 
    get_line_info,
    add_br_outside_blocks, 
    convert_markdown_to_html
)

In [2]:
import spacy
nlp = spacy.load("fr_core_news_md")

In [3]:
async def parse_user_query(
    query: str, 
    synonymes_groupes: dict[str, list[str]], 
    simple_dict: list[dict]) -> dict[str, list[str]]:
    """Transforme une question utilisateur en paramètres de fonction."""
    def _strip_accents(text: str) -> str:
        """
        Supprime les accents et normalise la casse/ponctuation dans une chaîne pour faciliter les comparaisons.
        """
        if not isinstance(text, str) or not text.strip():
            return ""
        s = (
            text.replace("’", "'")
                .replace("`", "'")
                .replace("‘", "'")
                .replace("–", "-")
                .replace("—", "-")
        )
        s = unicodedata.normalize("NFD", s)
        s = ''.join(
            c for c in s
            if unicodedata.category(c) != 'Mn'
        )
        s = re.sub(r"\s+", " ", s.lower()).strip()
        return s

    async def _check_detail(question: str, threshold: int = 80) -> bool:
        question_norm = _strip_accents(question.lower())
        res = await get_mapping()
        keywords_details: list[str] = res["Détail"]
        for kw in keywords_details:
            if fuzz.partial_ratio(question_norm, _strip_accents(kw.lower())) > threshold:
                return True
        return False

    async def _get_col_info(question: str, threshold: int = 90) -> dict|None:
        lst = await execute_sp(
            "dbo.sp_simBudCol",
            {
                "user_fk": 8,
                "codeMetier": 'EXP',
                "form_fk": 167,
                "codeFormType": None,
                "type_fk": 0,
                "colYear_fk": 0
            }
        )
        df_col = pd.DataFrame(lst)
        df_col = df_col[df_col["labelType"]=="Année contexte"][["label", "RB", "Mois", "theYear"]].copy()
        
        q = _strip_accents(question.lower())
        q_tokens = set(re.findall(r"\w+", q))
        LISTE_LIGNES: list[str] = df_col["label"].to_list()
        results = {}
        for ligne in LISTE_LIGNES:
            ln = _strip_accents(ligne.lower())
            ln_tokens = set(re.findall(r"\w+", ln))
            if ln_tokens and ln_tokens.issubset({str(t).rstrip('s') for t in q_tokens} | q_tokens):
                results[ligne] = 100
                continue
            score = fuzz.token_set_ratio(q, ln)
            if score >= threshold:
                results[ligne] = max(results.get(ligne, 0), score)
        if results:
            line = df_col[df_col["label"] == sorted(results.items(), key=lambda x: -x[1])[0][0]]
            return {
                "label": line["label"].iloc[0],
                "annee": int(line["theYear"].iloc[0]),
                "mois": int(line["Mois"].iloc[0]),
                "contexte": line["RB"].iloc[0]
            }    
        else:
            None

    doc = nlp(query)
    query_lower = query.lower()
    params = {
        'groupes': [],
        'types_valeur': [],
        'annees': [],
        'nature_ecriture': [],
        'lignes': [],
        'mois': []
    }
    col_infos = await _get_col_info(query_lower)

    # GROUPE
    def _select_groupe(question: str, threshold: float = 70) -> list:
        query_norm = _strip_accents(question)

        best_matches: dict[str, float] = {}
        max_overall_score = 0.0

        for g, mots in list(synonymes_groupes.items())[:3]:
            max_group_score = 0.0
            for m in mots:
                m_norm = _strip_accents(m)
                score = fuzz.token_set_ratio(query_norm, m_norm)
                if score > max_group_score:
                    max_group_score = score
                    
            best_matches[g] = max_group_score
            
            if max_group_score > max_overall_score:
                max_overall_score = max_group_score
        res = []
        if max_overall_score >= threshold:
            score_tolerance = 5
            for g, score in best_matches.items():
                if score >= threshold and score >= (max_overall_score - score_tolerance):
                    res.append(g)
        return res
    params['groupes'] = _select_groupe(query_lower)

    # TYPE DE VALEUR
    def _define_contexte(question: str) -> list:
        types_valeur = set()
        query_lower_noacc = _strip_accents(question)
        if re.search(r"reel|realise|actuel", query_lower_noacc):
            types_valeur.add('R')
        if re.search(r"budget", query_lower_noacc):
            types_valeur.add('B')
        if re.search(r"prevision|prevu|projection|project|prev", query_lower_noacc):
            types_valeur.add('P')
        for ent in doc.ents:
            ent_text_noacc = _strip_accents(ent.text.lower())
            if ent.label_ == "MISC":
                if re.search(r"reel|realise|actuel", ent_text_noacc):
                    types_valeur.add('R')
                if re.search(r"budget", ent_text_noacc):
                    types_valeur.add('B')
                if re.search(r"prevision|prevu|projection|project|prev", ent_text_noacc):
                    types_valeur.add('P')
        return sorted(types_valeur)
    params['types_valeur'] = _define_contexte(query_lower)

    # ANNEE
    def _define_year(question: str) -> list:
        annees = set(map(int, re.findall(r"\b20\d{2}\b", question)))
        
        pattern_2digit = r"\b(?:Budget|Réel|Reel)\s*'?(\d{2})\b"
        matches_2digit = re.findall(pattern_2digit, question, re.IGNORECASE)
        for match in matches_2digit:
            yy = int(match)
            if yy < 50:
                year = 2000 + yy
            else:
                year = 1900 + yy
            annees.add(year)

        for ent in doc.ents:
            if ent.label_ == "DATE":
                if ent.text.isdigit() and len(ent.text) == 4 and ent.text.startswith("20"):
                    annees.add(int(ent.text))
                elif re.match(r"20\d{2}[-/ ]20\d{2}", ent.text):
                    annees.update(map(int, re.findall(r"20\d{2}", ent.text)))
                else:
                    annees.update(map(int, re.findall(r"20\d{2}", ent.text)))

                ent_matches = re.findall(pattern_2digit, ent.text, re.IGNORECASE)
                for em in ent_matches:
                    yy = int(em)
                    if yy < 50:
                        year = 2000 + yy
                    else:
                        year = 1900 + yy
                    annees.add(year)

        interval_patterns_annee = [
            r"(?:entre)\s+20(\d{2})\s+(?:et)\s+20(\d{2})",
            r"(?:de)\s+20(\d{2})\s+(?:à|a)\s+20(\d{2})"
        ]
        for pat in interval_patterns_annee:
            m = re.search(pat, question)
            if m:
                y1, y2 = int("20" + m.group(1)), int("20" + m.group(2))
                if y1 <= y2:
                    annees.update(range(y1, y2 + 1))
                else:
                    annees.update(range(y2, y1 + 1))

        if annees:
            return sorted(annees)
        else:
            if re.search(r"cette (année|annee)", question):
                return [datetime.now().year]
            elif re.search(r"(année|annee) dernière|an dernier|(année|annee) (passée|passee)|(année|annee) (précédente|precedente)", question):
                return [datetime.now().year - 1]
            elif re.search(r"(l'année|l'annee) prochaine|l'an prochain", question):
                return [datetime.now().year + 1]
    params['annees'] = _define_year(query_lower)

    # MOIS
    def _define_month(question: str) -> list:
        mois_map = {
            1: [r"janv(?:ier)?", r"jan"],
            2: [r"f[ée]vr(?:ier)?", r"fev"],
            3: [r"mars?", r"mar"],
            4: [r"avr(?:il)?", r"avr"],
            5: [r"mai"],
            6: [r"juin"],
            7: [r"juil(?:let)?", r"jul"],
            8: [r"ao[uû]t?", r"aou"],
            9: [r"sept(?:embre)?", r"sep"],
            10: [r"oct(?:obre)?", r"oct"],
            11: [r"nov(?:embre)?", r"nov"],
            12: [r"d[ée]c(?:embre)?", r"dec"],
        }

        query_norm = _strip_accents(question)
        mois = set()
        interval_patterns = [
            r"(?:entre)\s+([a-zéûî\.]+)\s+(?:et)\s+([a-zéûî\.]+)",
            r"(?:de)\s+([a-zéûî\.]+)\s+(?:à|a)\s+([a-zéûî\.]+)",
        ]
        interval_match = None
        for pat in interval_patterns:
            m = re.search(pat, query_norm)
            if m:
                interval_match = m
                break

        if interval_match:
            mois1_txt, mois2_txt = interval_match.groups()
            mois1_txt = mois1_txt.replace('.', '')
            mois2_txt = mois2_txt.replace('.', '')
            mois1_num, mois2_num = None, None
            for num, patterns in mois_map.items():
                for pat in patterns:
                    if re.fullmatch(pat, mois1_txt):
                        mois1_num = num
                    if re.fullmatch(pat, mois2_txt):
                        mois2_num = num
            if mois1_num and mois2_num:
                if mois1_num <= mois2_num:
                    mois = set(range(mois1_num, mois2_num + 1))
                else:
                    mois = set(list(range(mois1_num, 13 + 1)) + list(range(1, mois2_num + 1)))
        else:
            for num, patterns in mois_map.items():
                for pat in patterns:
                    if re.search(rf"\b{pat}\b", query_norm):
                        mois.add(num)

            trimestre_regex = [
                (r"\b(1(er)?|premier|i+)[s\-]*(trimestre|trim)\b", [1, 2, 3]),
                (r"\b(2(e|ème|eme)?|deuxi[eè]me|ii+)[s\-]*(trimestre|trim)\b", [4, 5, 6]),
                (r"\b(3(e|ème|eme)?|troisi[eè]me|iii+)[s\-]*(trimestre|trim)\b", [7, 8, 9]),
                (r"\b(4(e|ème|eme)?|quatri[eè]me|iv+)[s\-]*(trimestre|trim)\b", [10, 11, 12]),
            ]
            for pat, mois_list in trimestre_regex:
                if re.search(pat, query_norm):
                    mois.update(mois_list)
                    break

            semestre_regex = [
                (r"\b(1(er)?|premier|i+)[s\-]*(semestre|sem)\b", [1, 2, 3, 4, 5, 6]),
                (r"\b(2(e|ème|eme)?|deuxi[eè]me|ii+)[s\-]*(semestre|sem)\b", [7, 8, 9, 10, 11, 12]),
            ]
            for pat, mois_list in semestre_regex:
                if re.search(pat, query_norm):
                    mois.update(mois_list)
                    break

            if re.search(r"\btous les mois\b", query_norm):
                mois.update(range(1, 13))
            elif re.search(r"\bmois courant\b", query_norm):
                mois.update([datetime.now().month])
            elif re.search(r"\bmois dernier\b", query_norm):
                mois.update([datetime.now().month - 1 if datetime.now().month > 1 else 12])
            elif re.search(r"\bmois prochain\b", query_norm):
                mois.update([datetime.now().month + 1 if datetime.now().month < 12 else 1])
        return sorted(mois)
    params['mois'] = _define_month(query_lower)

    # NATURE ECRITURE
    def _select_nature_ecriture(question: str) -> list:
        nature_ecritures = set()
        if re.search(r"\b(mensuel(le)?|mois|trimestre|semestre|janvier|février|fevrier|mars|avril|mai|juin|juillet|août|aout|septembre|octobre|novembre|décembre|decembre)\b", question):
            nature_ecritures.add('Mensuelle')
        if re.search(r"\b(annuel(le)?|total|cette année|cette annee|année|annee)\b", question):
            nature_ecritures.add('Annuelle')
        if not nature_ecritures:
            if params.get('mois') and 0 < len(params['mois']) < 12:
                nature_ecritures.add('Mensuelle')
            if params.get('mois') and len(params['mois']) == 12:
                nature_ecritures.add('Annuelle')
        return [v for v in ['Mensuelle', 'Annuelle'] if v in nature_ecritures]
    params['nature_ecriture'] = _select_nature_ecriture(query_lower)

    # LIGNES
    async def _match_lignes(question: str, threshold: int = 75, return_scores: bool = False):
        result_list = extract_all_descendants_for_list(simple_dict)
        LISTE_LIGNES: list[str] = [label for sublist in result_list for label in sublist]
        q = _strip_accents(question.lower())
        q_tokens = set(re.findall(r"\w+", q))
        results = {}
        # Plus grande tolérance au singulier/pluriel : on compare formes singulier et pluriel, pour chaque token de la ligne et de la question
        def _sing_plur_forms(token):
            if token.endswith('s'):
                return {token, token[:-1]}
            else:
                return {token, token + 's'}
        
        for ligne in LISTE_LIGNES:
            ln = _strip_accents(ligne.lower())
            ln_tokens_raw = set(re.findall(r"\w+", ln))
            q_tokens_raw = q_tokens

            # Génère toutes formes singulier/pluriel pour ln_tokens et q_tokens
            ln_tokens_all = set()
            for t in ln_tokens_raw:
                ln_tokens_all.update(_sing_plur_forms(t))
            q_tokens_all = set()
            for t in q_tokens_raw:
                q_tokens_all.update(_sing_plur_forms(t))
            
            # Test : Tous les tokens 'ligne' (sing/plur) présents dans q (sing/plur)
            if ln_tokens_all and ln_tokens_all.issubset(q_tokens_all):
                results[ligne] = 100
                continue

            # Fallback fuzzy
            score = fuzz.token_set_ratio(q, ln)
            if score >= threshold:
                results[ligne] = max(results.get(ligne, 0), score)

        ordered = sorted(results.items(), key=lambda x: -x[1])
        detail = await _check_detail(question)
        if detail and ordered:
            enfants = []
            for l in ordered:
                enfants += get_children_by_label(simple_dict, l[0])
            return set(enfants + [lbl for lbl, _ in ordered])
        return ordered if return_scores else [lbl for lbl, _ in ordered]
    params['lignes'] = await _match_lignes(query_lower)

    # Correct for NoneType issues and robustify list usage
    if col_infos:
        annee = col_infos.get("annee")
        # Ensure params['annees'] is always a list
        if not isinstance(params.get('annees'), list) or params['annees'] is None:
            params['annees'] = []
        if annee is not None and annee not in params['annees']:
            params['annees'].append(annee)
        mois = col_infos.get("mois", 0)
        if not isinstance(params.get('nature_ecriture'), list) or params['nature_ecriture'] is None:
            params['nature_ecriture'] = []
        if mois == 0:
            if "Annuelle" not in params['nature_ecriture']:
                params['nature_ecriture'].append("Annuelle")
        else:
            if "Mensuelle" not in params['nature_ecriture']:
                params['nature_ecriture'].append("Mensuelle")
            if not isinstance(params.get('mois'), list) or params['mois'] is None:
                params['mois'] = []
            if mois not in params['mois']:
                params['mois'].append(mois)

        contexte = col_infos.get("contexte")
        if not isinstance(params.get('types_valeur'), list) or params['types_valeur'] is None:
            params['types_valeur'] = []
        if contexte and contexte not in params['types_valeur']:
            params['types_valeur'].append(contexte)

    # Normalize all params lists (ensure always list, de-duplicate, and sort if non-empty)
    for k in ['groupes', 'types_valeur', 'annees', 'nature_ecriture', 'lignes', 'mois']:
        param_val = params.get(k)
        if not isinstance(param_val, list) or param_val is None:
            params[k] = []
        else:
            params[k] = sorted(set(param_val)) if param_val else []

    if not params['nature_ecriture']:
        params['nature_ecriture'] = ['Annuelle']

    if not params['annees']:
        params['annees'] = [datetime.now().year]

    return params

In [4]:
async def get_data_for_llm(
    df: pd.DataFrame,
    simple_dict: list[dict],
    sa_fk: int,
    form_fk: int,
    types_valeur: list[str],
    nature_ecriture: list[str],
    groupes: list[str] = None,
    mois: list = None,
    annees: list = None,
    lignes: list[str] = None) -> str:
    """
    Args:
        df (pd.DataFrame): DataFrame des écritures catégorisées à filtrer.
        simple_dict (list[dict]): Arbre hiérarchique des lignes/postes.
        types_valeur (list[str]): Types de valeur à inclure (ex: ['R', 'B']).
        nature_ecriture (list[str]): Types d'écriture à inclure (ex: ['Annuelle', 'Mensuelle']).
        groupes (list[str], optional): Groupes analytiques à filtrer. None = tous.
        mois (list[int], optional): Mois à inclure (ex: [1,2,3]). None = tous.
        annees (list[int], optional): Années à inclure (ex: [2024,2025]). None = toutes.
        lignes (list[str], optional): Postes/écritures à inclure. None = tous.

    Returns:
        is_not_empty (boolean): 
        df_markdown (str): Tableau Markdown filtré pour le prompt, ou texte vide si aucun résultat.
    """
    df_temp: pd.DataFrame = df.copy()
    for col in ['Montant', 'Année', 'Mois']:
        if col in df_temp.columns:
            df_temp[col] = pd.to_numeric(df_temp[col], errors='coerce')
    df_temp['Groupe'] = df_temp['Groupe'].fillna('')

    filtres = (
        df_temp['Contexte'].str.lower().isin([t.lower() for t in types_valeur]) &
        df_temp["Nature de l'écriture"].str.lower().isin([n.lower() for n in nature_ecriture])
    )

    if annees:
        filtres &= df_temp['Année'].isin([int(a) for a in annees])

    if mois and len(mois) > 0:
        mois = [int(m) for m in mois]
        filtres = (
            filtres &
            (
                ((df_temp["Nature de l'écriture"].str.lower() == "mensuelle") & df_temp["Mois"].isin(mois)) |
                (df_temp["Nature de l'écriture"].str.lower() == "annuelle")
            )
        )
    
    filtres_hierarchie = df_temp['Code Hiérarchique'].apply(
        lambda x: isinstance(x, str) and x.count('.') <= 2 # ! Code hierarchique premier parent et enfant
    )

    result_list = extract_all_descendants_for_list(simple_dict)
    CHIFFRE_AFFAIRES = result_list[0] + result_list[3] + result_list[6] + result_list[9] + result_list[10]
    CHARGES = result_list[1] + result_list[4] + result_list[7] + result_list[12]
    MARGES = result_list[2] + result_list[5] + result_list[8] + result_list[11] + result_list[13]

    if lignes and len(lignes) > 0 and groupes and len(groupes) > 0:
        lignes_valides_du_groupe: list[str] = []
        for groupe in groupes:
            if groupe == "Chiffre d'affaire":
                lignes_valides_du_groupe += CHIFFRE_AFFAIRES
            elif groupe == "Charge":
                lignes_valides_du_groupe += CHARGES
            elif groupe == "Marge":
                lignes_valides_du_groupe += MARGES
        
        toutes_lignes_coherentes = all(l in lignes_valides_du_groupe for l in lignes)
        
        if toutes_lignes_coherentes:
            filtres_lignes = df_temp["Lignes"].str.lower().isin([l.lower() for l in lignes])
            filtres &= filtres_lignes
        else:
            filtres_groupes = df_temp['Groupe'].str.lower().isin([g.lower() for g in groupes])
            filtres &= filtres_groupes
    elif lignes and len(lignes) > 0:
        filtres_lignes = df_temp["Lignes"].str.lower().isin([l.lower() for l in lignes])
        filtres &= filtres_lignes
    elif groupes and len(groupes) > 0:
        filtres_groupes = df_temp['Groupe'].str.lower().isin([g.lower() for g in groupes])
        filtres &= filtres_groupes
        filtres &= filtres_hierarchie
    else:
        filtres &= filtres_hierarchie

    df_filtre: pd.DataFrame = df_temp.loc[filtres].copy()

    all_annu = all(n.lower() == "annuelle" for n in nature_ecriture)
    all_mensu = all(n.lower() == "mensuelle" for n in nature_ecriture)

    def _code_to_tuple(code):
        if pd.isna(code):
            return (999999,)
        s = str(code).strip()
        if s.endswith('.'):
            s = s.rstrip('.')
        nums = re.findall(r'\d+', s)
        if not nums:
            return (999999,)
        return tuple(int(n) for n in nums)

    def _filter_constant_columns(cols: list):
        EXCLUDE = {"Lignes", "Montant"}
        if df_filtre is not None and len(df_filtre) > 0:
            constant_columns = cols.copy()
            for col_name in constant_columns:
                if col_name in EXCLUDE:
                    continue
                if col_name in df_filtre.columns and df_filtre[col_name].nunique() == 1:
                    cols = [c for c in cols if c != col_name]
        return cols

    def _flatten_labels(simple_dict: list[dict]) -> pd.DataFrame:
        def __flatten_labels(simple_dict: list[dict]) -> list[dict]:
            flat_list = []
            for item in simple_dict:
                label = item.get("label", "")
                code = item.get("code", "")
                flat_list.append({"code": code, "label": label})
                children = item.get("children", [])
                if children:
                    flat_list.extend(__flatten_labels(children))
            return flat_list
        return pd.DataFrame(__flatten_labels(simple_dict), columns=["code", "label"])

    def _code_parts_raw(code: str) -> list:
        if not isinstance(code, str):
            return []
        parts = code.split('.')
        parts = [p for p in parts if p != ""]
        return parts

    def _find_siblings_with_neighbors(
        label: str, 
        df_labels_codes: pd.DataFrame, 
        n_before: int = config.N_NEIGHBORS, 
        n_after: int = config.N_NEIGHBORS) -> list[str]:

        row_mask = df_labels_codes["label"] == label
        if not row_mask.any():
            return []

        idx = df_labels_codes.index[row_mask][0]
        code_central = df_labels_codes.at[idx, "code"]
        central_parts = _code_parts_raw(code_central)

        if len(central_parts) <= 1:
            def __is_root(c):
                return len(_code_parts_raw(c)) == 1
            df_roots = df_labels_codes[df_labels_codes["code"].apply(__is_root)].copy()
            label_idx_in_roots = df_roots.index[df_roots["label"] == label]
            if label_idx_in_roots.empty:
                return df_roots["label"].to_list()
            i = label_idx_in_roots[0]
            start = max(0, i - n_before)
            end = i + n_after + 1
            sel = df_roots.iloc[start:end]
            return sel["label"].to_list()

        parent_parts = central_parts[:-1]
        target_depth = len(central_parts)

        def __is_same_parent(c):
            parts = _code_parts_raw(c)
            if len(parts) != target_depth:
                return False
            cand_parent = parts[:-1]
            return cand_parent == parent_parts

        df_siblings = df_labels_codes[df_labels_codes["code"].apply(__is_same_parent)].copy()
        label_idx_in_siblings = df_siblings.index[df_siblings["label"] == label]
        if label_idx_in_siblings.empty:
            return df_siblings.reset_index(drop=True)["label"].to_list()
        i = label_idx_in_siblings[0]
        i_relative = list(df_siblings.index).index(i)
        start = max(0, i_relative - n_before)
        end = i_relative + n_after + 1
        sel: pd.DataFrame = df_siblings.iloc[start:end]
        return sel["label"].to_list()

    def _find_siblings_with_col(
        annee: int, 
        mois: int,
        contexte: str,
        df_col: pd.DataFrame, 
        n_before: int = config.N_NEIGHBORS, 
        n_after: int = config.N_NEIGHBORS) -> str:

        if df_col is None or df_col.empty:
            return ""

        mask = (df_col["theYear"] == annee) & (df_col["RB"].str.lower() == contexte.lower())
        if mois == 0:
            mask = mask & (df_col["Mois"] == 0)
        else:
            mask = mask & (df_col["Mois"] == mois)
        idx_list = df_col[mask].index.tolist()
        if not idx_list:
            return ""
        idx = idx_list[0]

        if mois == 0:
            mois0_indices = df_col[df_col["Mois"] == 0].index.tolist()
            try:
                pos_in_mois0 = mois0_indices.index(idx)
            except ValueError:
                return ""
            start = max(0, pos_in_mois0 - n_before)
            end = pos_in_mois0 + n_after + 1
            indices = mois0_indices[start:end]
        else:
            year_contexte_indices = df_col[(df_col["theYear"] == annee) & (df_col["RB"].str.lower() == contexte.lower()) & (df_col["Mois"] != 0)].index.tolist()
            try:
                pos_in_year_contexte = year_contexte_indices.index(idx)
            except ValueError:
                return ""
            start = max(0, pos_in_year_contexte - n_before)
            end = pos_in_year_contexte + n_after + 1
            indices = year_contexte_indices[start:end]

        if not indices:
            return ""

        try:
            rows = df_col.loc[indices, ["theYear", "Mois", "RB"]]
        except KeyError:
            return ""
        cols_labels = ",".join([f"{int(row.theYear)},{int(row.Mois)},{row.RB}" for _, row in rows.iterrows()])
        return cols_labels

    def _get_line_fk(label: str, res: list[dict]):
        df = pd.DataFrame(res).iloc[:, [0, 4]]
        ser: pd.DataFrame = df[df["label"] == label]["line_id"]
        return int(ser.iloc[0])

    if all_annu:
        colonnes_finales = ['Code Hiérarchique', 'Lignes', 'Année', 'Contexte', 'Montant']
        colonnes_tri = ['Code Hiérarchique', 'Année', 'Lignes']
    elif all_mensu:
        colonnes_finales = ['Code Hiérarchique', 'Lignes', 'Mois', 'Année', 'Contexte', 'Montant']
        colonnes_tri = ['Code Hiérarchique', 'Année', 'Mois', 'Lignes']
    else:
        colonnes_finales = ['Code Hiérarchique', 'Lignes', 'Mois', 'Année', 'Nature de l\'écriture', 'Contexte', 'Montant']
        colonnes_tri = ['Code Hiérarchique', 'Année']
        if 'Mois' in df_filtre.columns:
            colonnes_tri.append('Mois')
        colonnes_tri.extend(['Nature de l\'écriture', 'Lignes'])


    colonnes_finales = _filter_constant_columns(colonnes_finales)
    colonnes_tri = _filter_constant_columns(colonnes_tri)
    tri_effectif = [col for col in colonnes_tri if col in df_filtre.columns]

    if 'Code Hiérarchique' in tri_effectif and 'Code Hiérarchique' in df_filtre.columns:
        df_filtre['_code_sort'] = df_filtre['Code Hiérarchique'].apply(_code_to_tuple)
        tri_effectif = ['_code_sort' if c == 'Code Hiérarchique' else c for c in tri_effectif]

    colonnes_presentes = [col for col in colonnes_finales if col in df_filtre.columns]

    need_code_sort = '_code_sort' in tri_effectif and '_code_sort' not in colonnes_presentes
    if need_code_sort:
        cols_for_sort = ['_code_sort'] + colonnes_presentes
    else:
        cols_for_sort = colonnes_presentes

    df_final = df_filtre[cols_for_sort].sort_values(by=tri_effectif).reset_index(drop=True)
    if need_code_sort:
        df_final = df_final.drop(columns=['_code_sort'])

    # * Case: Line empty * #
    if lignes and list(lignes)[0] not in df_final["Lignes"].to_list():
        if not types_valeur:
            return False, None

        res = await execute_sp(
            "dbo.sp_simBudLines",
            {
                "user_fk": config.USER_FK,
                "form_fk": form_fk,
                "line_fk": 0,
                "choix": 0,
                "isVisible": 1
            }
        )
        df_labels_codes = _flatten_labels(simple_dict)

        colonne_db = await execute_sp(
            "dbo.sp_simBudCol",
            {
                "user_fk": config.USER_FK,
                "codeMetier": 'EXP',
                "form_fk": form_fk,
                "codeFormType": None,
                "type_fk": 0,
                "colYear_fk": 0
            }
        )
        df_col = pd.DataFrame(colonne_db)
        df_col = df_col[df_col["labelType"]=="Année contexte"][["label", "RB", "Mois", "theYear"]].copy()
        lines = _find_siblings_with_neighbors(
            label=lignes[0], 
            df_labels_codes=df_labels_codes
        )
        mois_val = mois[0] if mois and len(mois) > 0 else 0
        cols = _find_siblings_with_col(
            annee=annees[0],
            mois=mois_val,
            contexte=types_valeur[0],
            df_col=df_col
        )

        line_fks = [str(_get_line_fk(l, res)) for l in lines]
        line_fks_str = ",".join(line_fks)
        the_line_fk = _get_line_fk(lignes[0], res)
        
        lst = await execute_sp(
            "ia.sp_simBudValueDetails",
            {
                "user_fk": config.USER_FK,
                "listSA": 0,
                "line_fk": line_fks_str,
                "sa_fk": sa_fk,
                "yearRB": cols,
            }
        )
        df_result = pd.DataFrame(lst)
        
        if df_result.empty:
            return False, None
        
        line_info = get_line_info(
            df_result, 
            the_line_fk, 
            annees[0],
            mois_val,
            types_valeur[0],
            lines,
            lambda label: _get_line_fk(label=label, res=res)
        )

        col_info = get_col_info(
            df_result, 
            the_line_fk,
            cols
        )
        return False, (
            f"La ligne '{lignes[0]}' n'a aucune valeur. "
            "Analyse les deux tableaux ci-dessous :\n"
            " - le premier montre les lignes voisines et leurs sources\n"
            " - le second montre les colonnes voisines et leurs sources\n"
            "Explique très brièvement pourquoi cette ligne n'a aucune donnée. "
            "Compare uniquement avec les voisins (lignes et colonnes) pour justifier l'absence. "
            "Si les colonnes voisines pour cette même ligne contiennent des sources, spécifie-les (année, mois, contexte, source). "
            "Réponse 1-3 phrases, sans paragraphes, directe.\n"
            f"{line_info}\n\n{col_info}"
        )
    
    if df_final.empty:
        return False, None
    else:
        return True, df_final #.to_markdown(index=False)


In [5]:
res = await execute_sp(
    "dbo.sp_simBudLines",
    {
        "user_fk": config.USER_FK,
        "form_fk": 167,
        "line_fk": 0,
        "choix": 0,
        "isVisible": 1
    }
)
simple_dict = create_simplified_hierarchy(res)
lexiques = await get_mapping()

In [6]:
df = pd.read_csv(r"data.csv")
df = preprocessing_data(df, simple_dict)
df.head()


,Section analytique,Liste de sélection,Formulaire,Code Hiérarchique,Lignes,Type de lignes,Compte reporting,Donnée opérationnelle,Colonnes,Type de colonnes,Montant,Cumul,Nature de l'écriture,Date,Année,Mois,Contexte,Groupe
0,11000099 - Noisy,NaN,Compte d'exploitation,4,% DES RECETTES TOTALES,Ligne de calcul,NaN,NaN,Réel 2022,Année contexte,95,0.0,Mensuelle,2022-11-30,2022.0,11.0,R,Chiffre d'affaire
1,11000099 - Noisy,NaN,Compte d'exploitation,7,% DES RECETTES TOTALES,Ligne de calcul,NaN,NaN,Prév 2025,Année contexte,87,0.0,Mensuelle,2025-08-31,2025.0,8.0,P,Chiffre d'affaire
2,11000099 - Noisy,NaN,Compte d'exploitation,10,% DES RECETTES TOTALES,Ligne de calcul,NaN,NaN,Réel 2024,Année contexte,87,0.0,Mensuelle,2024-08-31,2024.0,8.0,R,Chiffre d'affaire
3,11000099 - Noisy,NaN,Compte d'exploitation,10,% DES RECETTES TOTALES,Ligne de calcul,NaN,NaN,Budget 2026,Année contexte,85,0.0,Mensuelle,2026-05-31,2026.0,5.0,B,Chiffre d'affaire
4,11000099 - Noisy,NaN,Compte d'exploitation,10,% DES RECETTES TOTALES,Ligne de calcul,NaN,NaN,Budget 2025,Année contexte,85,0.0,Mensuelle,2025-05-31,2025.0,5.0,B,Chiffre d'affaire


In [7]:
df.to_csv(r"data_pretraited.csv", index=False)

In [8]:
question = "Quel est le coût total des charges pour le mois de novembre 2025 ?"
parametres = await parse_user_query(question, lexiques, simple_dict)
parametres

{'groupes': ['Charge'],
 'types_valeur': [],
 'annees': [2025],
 'nature_ecriture': ['Annuelle', 'Mensuelle'],
 'lignes': ['Total 1', 'Total 2'],
 'mois': [11]}

In [9]:
parametres = {
    'groupes': ['Charge'],
    'types_valeur': ['R', 'B', 'P'],
    'annees': [datetime.now().year-1, datetime.now().year],
    'nature_ecriture': ['Annuelle'],
    'lignes': [],
    'mois': []
}

parametres

{'groupes': ['Charge'],
 'types_valeur': ['R', 'B', 'P'],
 'annees': [2024, 2025],
 'nature_ecriture': ['Annuelle'],
 'lignes': [],
 'mois': []}

In [10]:
s, df_res = await get_data_for_llm(df, simple_dict, 224, 167, **parametres)
df_res

,Code Hiérarchique,Lignes,Année,Contexte,Montant
0,2.,CHARGES D'IMMEUBLE DIRECTES,2024.0,R,663916
1,2.,CHARGES D'IMMEUBLE DIRECTES,2025.0,B,650619
2,2.,CHARGES D'IMMEUBLE DIRECTES,2025.0,P,666871
3,2.1.,FRAIS DE PERSONNEL,2024.0,R,168150
4,2.1.,FRAIS DE PERSONNEL,2025.0,B,202380
...,...,...,...,...,...
72,8.3.,HONORAIRES SYNDIC,2024.0,R,25342
73,8.3.,HONORAIRES SYNDIC,2025.0,B,26102
74,8.3.,HONORAIRES SYNDIC,2025.0,P,25532
75,13.,CAPEX,2024.0,R,15964


In [11]:
def analyze_costs(
    df: pd.DataFrame,
    annee: int,
    groupe: str = None
) -> Dict[str, Any]:
    
    # ---- 1. Filtrage selon critères ----
    dff = df[df["Année"] == annee]

    if groupe:
        dff = dff[dff["Groupe"] == groupe]

    dff = dff[dff["Nature de l'écriture"] == "Annuelle"]
    
    # ---- 2. Déterminer les lignes FEUILLES pour éviter la double addition ----
    # Toutes les lignes hiérarchiques
    all_codes = dff["Code Hiérarchique"].astype(str).tolist()
    branches_codes = [code for code in all_codes if code.count('.') == 1 and code.endswith('.')]

    # Filtrer uniquement les feuilles
    dff = dff[dff["Code Hiérarchique"].astype(str).isin(list(set(branches_codes)))]
    print(list(set(branches_codes)))
    # ---- 3. Extraction P, B, R ----
    def get_value(context):
        res = dff[dff["Contexte"] == context]["Montant"].sum()
        return float(res) if not pd.isna(res) else 0.0

    prevision = get_value("P")
    budget = get_value("B")
    reel = get_value("R")

    # ---- 4. Calcul des écarts ----
    ecart_P_R = prevision - reel
    ecart_B_R = budget - reel
    ecart_P_B = prevision - budget

    # ---- 5. Taux d’exécution du budget ----
    taux_exec = (reel / budget * 100) if budget != 0 else None

    # ---- 6. Conclusion automatique ----
    conclusion = []

    if taux_exec is not None:
        if taux_exec > 110:
            conclusion.append("Dépassement budgétaire important.")
        elif taux_exec > 100:
            conclusion.append("Dépassement léger du budget.")
        elif taux_exec < 90:
            conclusion.append("Sous-exécution significative du budget.")
        else:
            conclusion.append("Exécution budgétaire maîtrisée.")

    if abs(ecart_P_R) > 0.1 * (reel if reel != 0 else 1):
        conclusion.append("Écart prévisionnel significatif.")

    conclusion = " ".join(conclusion) or "Situation financière stable."

    # ---- 7. Structure finale ----
    return {
        "Analyse": groupe,
        "résidence": str(df.iloc[1,0]).split(' - ')[1],
        "année": annee,
        "prévision": int(round(prevision, 0)),
        "budget": int(round(budget, 0)),
        "réel": int(round(reel, 0)),
        "écart_P_R": int(round(ecart_P_R, 0)),
        "écart_B_R": int(round(ecart_B_R, 0)),
        "écart_P_B": int(round(ecart_P_B, 0)),
        "taux_exécution_budget": float(round(taux_exec, 2)),
        "conclusion": conclusion,
    }


In [12]:
r = analyze_costs(
    df,
    annee=datetime.now().year,
    groupe="Chiffre d'affaire"
)
md = pd.DataFrame([r]).to_markdown(index=False)
print(md)

['1.', '11.']
| Analyse           | résidence   |   année |   prévision |   budget |   réel |   écart_P_R |   écart_B_R |   écart_P_B |   taux_exécution_budget | conclusion                                                               |
|:------------------|:------------|--------:|------------:|---------:|-------:|------------:|------------:|------------:|------------------------:|:-------------------------------------------------------------------------|
| Chiffre d'affaire | Noisy       |    2025 |     2205492 |  2232944 |      0 |     2205492 |     2232944 |      -27452 |                       0 | Sous-exécution significative du budget. Écart prévisionnel significatif. |


In [13]:
prompt_system = (
    "Tu es un assistant français pour contrôleurs de gestion. "
    "Réponds de manière claire et concise. "
)

prompt = (
    "Analyse en deux ou trois phrases au maximum. \n"
    f"{md}"
)

In [14]:
messages = [
    {"role": "system", "content": prompt_system},
    {"role": "user", "content": prompt}
]

payload = {
    "model": config.GPT,
    "messages": messages,
    "stream": False,
    "keep_alive": -1,
    "options": {
        "temperature": 0.1
    }
}

In [15]:
with httpx.Client() as client:
    response = client.post(
        config.OLLAMA_URL,
        json=payload,
        timeout=httpx.Timeout(595)
    )
    response.raise_for_status()
    json_data = response.json()

if "message" in json_data and "content" in json_data["message"]:
    content: str = json_data["message"]["content"]
content

'Le chiffre d’affaires réel est nul alors que la prévision était de 2\u202f205\u202f492\u202f€ et le budget de 2\u202f232\u202f944\u202f€. L’écart prévisionnel est de –27\u202f452\u202f€ et l’écart budget‑réel de 2\u202f232\u202f944\u202f€, indiquant une sous‑exécution totale. Il faut identifier les causes de cette absence de revenus et réviser les prévisions.'

In [16]:
def format_response(content):
    content = convert_markdown_to_html(content)
    content = content.lstrip('\n')
    content = content.replace("\r\n", "\n")
    content = content.replace("\r", "\n")
    content = add_br_outside_blocks(content)
    content = re.sub(r"\*\*(.+?)\*\*", r"<b>\1</b>", content)
    content = re.sub(r"__(.+?)__", r"<b>\1</b>", content)
    content = re.sub(r"(?<!\*)\*(?!\*)(.+?)(?<!\*)\*(?!\*)", r"<i>\1</i>", content)
    content = re.sub(r"(?<!_)_(?!_)(.+?)(?<!_)_(?!_)", r"<i>\1</i>", content)
    content = re.sub(r"^<br/>\s*", "", content)
    content = re.sub(r"<br/>\s*$", "", content)
    return content

In [17]:
html_content = format_response(content)
display(HTML(html_content))

In [18]:
html_content_CA = format_response(content)
display(HTML(html_content_CA))

# Génération des chunks

In [19]:
import pandas as pd
import numpy as np

# Chargement du fichier (en supposant la même structure que précédemment)
df = pd.read_csv('data_pretraited.csv')

# Renommage et nettoyage
df = df.rename(
    columns={
        'Code Hiérarchique': 'Code_H', 
        'Montant': 'Montant',
        'Lignes': 'Ligne_Analytique',
        'Contexte': 'Contexte',
        'Année': 'Annee',
        'Groupe': 'Groupe',
        'Section  analytique': 'Residence'
    }
)

df['Montant'] = pd.to_numeric(df['Montant'], errors='coerce')
df['Annee'] = pd.to_numeric(df['Annee'], errors='coerce').astype('Int64')
df['Montant'] = df['Montant'].fillna(0).astype('Int64')

df_annuelle = df[df["Nature de l'écriture"] == "Annuelle"]
df_agg = df_annuelle.groupby(['Residence', 'Annee', 'Contexte', 'Code_H', 'Ligne_Analytique', 'Groupe'])['Montant'].sum().reset_index()

In [20]:
new_chunks = []
new_metadata = []

In [21]:
df_pivot = df_agg.pivot_table(
    index=['Residence', 'Annee', 'Code_H', 'Ligne_Analytique', 'Groupe'],
    columns='Contexte', 
    values='Montant', 
    fill_value=0
).reset_index()

# Renommage des colonnes pour les contextes
df_pivot = df_pivot.rename(columns={'R': 'Montant_R', 'P': 'Montant_P', 'B': 'Montant_B'})

In [22]:
# 1. Écart d'Objectif (Réel vs Budgété)
df_pivot['Ecart_R_B_Abs'] = df_pivot['Montant_R'] - df_pivot['Montant_B']
df_pivot['Ecart_R_B_Rel'] = np.where(
    df_pivot['Montant_B'] != 0, 
    df_pivot['Ecart_R_B_Abs'] / df_pivot['Montant_B'],
    0
)

# 2. Écart de Prévision (Réel vs Prévisionnel)
df_pivot['Ecart_R_P_Abs'] = df_pivot['Montant_R'] - df_pivot['Montant_P']

# 3. Écart de Révision (Prévisionnel vs Budgété)
df_pivot['Ecart_P_B_Abs'] = df_pivot['Montant_P'] - df_pivot['Montant_B']

In [24]:
def create_multi_context_chunk(row):
    # Variables de base
    residence = row['Residence']
    annee = int(row['Annee'])
    code_h = row['Code_H']
    ligne = row['Ligne_Analytique']
    groupe = row['Groupe']
    
    # Sens de l'écart
    is_charge = (groupe == 'Charge')
    
    # --- Analyse R vs B (Objectif) ---
    ecart_r_b_abs = row['Ecart_R_B_Abs']
    ecart_r_b_rel_pct = f"{row['Ecart_R_B_Rel'] * 100:.2f}%"
    
    if is_charge:
        performance_r_b = 'un dépassement' if ecart_r_b_abs > 0 else 'une économie'
        impact_r_b = 'DÉFAVORABLE' if ecart_r_b_abs > 0 else 'FAVORABLE'
    else:
        performance_r_b = 'une performance supérieure' if ecart_r_b_abs > 0 else 'un manque à gagner'
        impact_r_b = 'FAVORABLE' if ecart_r_b_abs > 0 else 'DÉFAVORABLE'
        
    # --- Analyse R vs P (Précision) ---
    ecart_r_p_abs = row['Ecart_R_P_Abs']
    precision_p = 'très précis' if abs(ecart_r_p_abs) < abs(row['Montant_R']) * 0.05 else 'imprécis' # Seuil arbitraire de 5%
    
    # --- Analyse P vs B (Révision) ---
    ecart_p_b_abs = row['Ecart_P_B_Abs']
    
    # --- Corps du Chunk ---
    chunk = (
        f"ANALYSE DE PERFORMANCE FINANCIÈRE | Résidence : {residence} | Année : {annee} | "
        f"Poste : {ligne} ({code_h}) | Type : {groupe}.\n\n"
        
        f"1. Écarts Réel (R) vs Budgété (B) :\n"
        f"   Montant Réel : {row['Montant_R']:,.2f} € | Montant Budgété : {row['Montant_B']:,.2f} €.\n"
        f"   L'écart s'élève à {ecart_r_b_abs:,.2f} € ({ecart_r_b_rel_pct}), ce qui représente {performance_r_b}. "
        f"L'impact net sur la Marge est **{impact_r_b}**.\n\n"
        
        f"2. Précision du Prévisionnel (P) :\n"
        f"   Montant Prévisionnel : {row['Montant_P']:,.2f} €.\n"
        f"   L'écart entre le Réel et la Prévision est de {ecart_r_p_abs:,.2f} €. Le Prévisionnel était {precision_p} "
        f"pour ce poste, indiquant une variation de {abs(ecart_r_p_abs):,.2f} € par rapport à l'anticipation.\n\n"
        
        f"3. Révision du Budget (P vs B) :\n"
        f"   Le Prévisionnel a été ajusté de {ecart_p_b_abs:,.2f} € par rapport au Budget initial, "
        f"reflétant une révision de l'objectif en cours d'année."
    )
    
    return chunk

# Génération des chunks
df_pivot['Chunk'] = df_pivot.apply(create_multi_context_chunk, axis=1)

chunks_list = df_pivot['Chunk'].tolist()
metadata_list = df_pivot[[
    'Residence', 'Annee', 'Code_H', 'Ligne_Analytique', 'Groupe', 
    'Ecart_R_B_Abs', 'Ecart_R_B_Rel', 'Ecart_R_P_Abs'
]].to_dict('records')

new_chunks.extend(chunks_list)
new_metadata.extend(metadata_list)

In [25]:
# 1. Préparation pour l'analyse des tendances (uniquement le contexte Réel)
df_reel_only = df_pivot.loc[df_pivot['Montant_R'].notna()].sort_values(by='Annee')

# 2. Calcul du montant Réel de l'année précédente (N-1)
# Utilisation de shift(1) groupé par résidence et ligne pour comparer N à N-1
df_reel_only['Montant_R_Prev'] = df_reel_only.groupby(['Residence', 'Code_H', 'Ligne_Analytique'])['Montant_R'].shift(1)

# 3. Calcul de la croissance absolue et relative N vs N-1
df_reel_only['Croissance_Abs'] = df_reel_only['Montant_R'] - df_reel_only['Montant_R_Prev']
df_reel_only['Croissance_Rel'] = np.where(
    df_reel_only['Montant_R_Prev'].fillna(0) != 0, 
    df_reel_only['Croissance_Abs'] / df_reel_only['Montant_R_Prev'],
    0
)

# Filtrer les lignes de la première année qui n'ont pas de N-1 pour la comparaison
df_yoy = df_reel_only.dropna(subset=['Montant_R_Prev']).copy()

In [26]:
def create_yoy_chunk(row):
    residence = row['Residence']
    annee_n = int(row['Annee'])
    annee_n_1 = annee_n - 1
    ligne = row['Ligne_Analytique']
    groupe = row['Groupe']
    
    croissance_abs = row['Croissance_Abs']
    croissance_rel_pct = f"{row['Croissance_Rel'] * 100:.2f}%"
    
    # Détermination du sens pour le LLM
    if croissance_abs > 0:
        direction = 'une AUGMENTATION'
        impact = 'NÉGATIF sur la marge' if groupe == 'Charge' else 'POSITIF'
    elif croissance_abs < 0:
        direction = 'une DIMINUTION'
        impact = 'POSITIF sur la marge' if groupe == 'Charge' else 'NÉGATIF'
    else:
        direction = 'une STAGNATION'
        impact = 'NEUTRE'

    chunk = (
        f"ANALYSE DE TENDANCE (RÉEL N vs N-1) | Résidence : {residence} | "
        f"Poste : {ligne} ({groupe}) | Période : {annee_n_1} vers {annee_n}.\n\n"
        f"**VALEURS CLÉS :**\n"
        f"   Réel {annee_n_1} : {row['Montant_R_Prev']:,.2f} €\n"
        f"   Réel {annee_n} : {row['Montant_R']:,.2f} €\n"
        f"   **Croissance Annuelle (YoY) : {direction} de {croissance_rel_pct}** ({croissance_abs:,.2f} €).\n"
        f"**IMPACT :** Cette tendance est considérée comme **{impact}** pour la résidence."
    )
    
    return chunk

# Génération et ajout des nouveaux chunks
df_yoy['Chunk_YoY'] = df_yoy.apply(create_yoy_chunk, axis=1)

# Fusion des listes de Chunks (Analyse Verticale + Analyse Horizontale)
yoy_chunks_list = df_yoy['Chunk_YoY'].tolist()
yoy_metadata_list = df_yoy[[
    'Residence', 
    'Annee', 
    'Code_H', 
    'Ligne_Analytique', 
    'Groupe', 
    'Croissance_Abs', 
    'Croissance_Rel'
]].rename(columns={
    # Renommer les colonnes d'écart pour la clarté dans la base vectorielle
    'Croissance_Abs': 'Ecart_YoY_Abs', 
    'Croissance_Rel': 'Ecart_YoY_Rel'
}).to_dict('records')
metadata_list += yoy_metadata_list

new_chunks.extend(yoy_chunks_list)
new_metadata.extend(yoy_metadata_list)

In [27]:
def create_analysis_chunk_and_metadata(row):
    # Extraction des variables principales
    residence = row.get('Residence', 'Inconnu')
    annee = int(row.get('Annee', 0))
    ligne = row.get('Ligne_Analytique', row.get('Ligne', ''))
    code_h = row.get('Code_H', '')
    groupe = row.get('Groupe', '')
    montant_r = row.get('Montant_R', 0.0) or 0.0
    montant_b = row.get('Montant_B', 0.0) or 0.0
    montant_p = row.get('Montant_P', 0.0) or 0.0

    # Calcul des écarts R vs B (réel vs budget)
    ecart_r_b_abs = montant_r - montant_b
    ecart_r_b_rel = (ecart_r_b_abs / montant_b) if montant_b not in (0, None) else 0.0
    ecart_r_b_rel_pct = f"{ecart_r_b_rel * 100:.2f}%"

    # Calcul des écarts R vs P (réel vs prévisionnel)
    ecart_r_p_abs = montant_r - montant_p
    ecart_r_p_rel = (ecart_r_p_abs / montant_p) if montant_p not in (0, None) else 0.0
    ecart_r_p_rel_pct = f"{ecart_r_p_rel * 100:.2f}%"

    # Impact marge pour R vs B
    if groupe.lower() == "charge":
        if ecart_r_b_abs > 0:
            impact_r_b = "DÉFAVORABLE"
        elif ecart_r_b_abs < 0:
            impact_r_b = "FAVORABLE"
        else:
            impact_r_b = "NEUTRE"
    else:  # recette, chiffre d'affaire, etc.
        if ecart_r_b_abs > 0:
            impact_r_b = "FAVORABLE"
        elif ecart_r_b_abs < 0:
            impact_r_b = "DÉFAVORABLE"
        else:
            impact_r_b = "NEUTRE"

    # Précision du prévisionnel (Fiabilité)
    if montant_p == 0:
        if montant_r == 0:
            precision_p = "parfaitement respecté (montant nul)"
        else:
            precision_p = "fortement sous-évalué"
    else:
        if abs(ecart_r_p_rel) < 0.05:
            precision_p = "très précis"
        elif ecart_r_p_rel > 0.05:
            precision_p = "sous-évalué"
        else:
            precision_p = "sur-évalué"

    # Structure du chunk
    chunk = (
        f"ANALYSE VERTICALE DÉTAILLÉE | Résidence : {residence} | Année : {annee} | "
        f"Poste : {ligne} (Code {code_h}) | Groupe : {groupe}.\n\n"
        
        f"**1. PERFORMANCE RÉEL vs. BUDGÉTÉ (R vs B) :**\n"
        f"   Montant Réel (R) : {montant_r:,.2f} € | Budgété (B) : {montant_b:,.2f} €.\n"
        f"   Écart Absolu : {ecart_r_b_abs:,.2f} € | Écart Relatif : {ecart_r_b_rel_pct}.\n"
        f"   **CONCLUSION : L'impact sur la Marge est {impact_r_b} par rapport à l'objectif budgétaire.**\n\n"
        
        f"**2. PRÉCISION RÉEL vs. PRÉVISIONNEL (R vs P) :**\n"
        f"   Montant Prévisionnel (P) : {montant_p:,.2f} €.\n"
        f"   Écart R vs P : {ecart_r_p_abs:,.2f} €.\n"
        f"   **CONTEXTE :** Le Prévisionnel était {precision_p} (écart de {abs(ecart_r_p_abs):,.2f} €), ce qui mesure la fiabilité de l'anticipation."
    )

    # Construction des métadonnées détaillées
    metadata = {
        "Residence": residence,
        "Annee": annee,
        "Ligne_Analytique": ligne,
        "Code_H": code_h,
        "Groupe": groupe,
        "Montant_R": montant_r,
        "Montant_B": montant_b,
        "Montant_P": montant_p,
        "Ecart_R_B_Abs": ecart_r_b_abs,
        "Ecart_R_B_Rel": ecart_r_b_rel,
        "Ecart_R_P_Abs": ecart_r_p_abs,
        "Ecart_R_P_Rel": ecart_r_p_rel,
        "Impact_Marge_R_B": impact_r_b,
        "Precision_Previ_P": precision_p
    }

    return chunk, metadata

vertical_chunk_and_meta = df_pivot.apply(create_analysis_chunk_and_metadata, axis=1)
vertical_chunks_list = [c for c, _ in vertical_chunk_and_meta]
vertical_metadata_list = [m for _, m in vertical_chunk_and_meta]

new_chunks.extend(vertical_chunks_list)
new_metadata.extend(vertical_metadata_list)


In [28]:
def calculer_ratios(df_pivot):
    # 1. Obtenir le CA (Chiffre d'Affaires) Réel, Budgété, et Prévisionnel par Résidence et Année
    df_ca = df_pivot[df_pivot['Groupe'] == "Chiffre d'affaire"].copy()
    
    # Si le CA est lui-même hiérarchisé, agréger le CA total (par exemple, Code_H '4' ou '5')
    # Pour la simplicité, on prend l'agrégat le plus élevé.
    ca_cols = ['Montant_R', 'Montant_P', 'Montant_B']
    ca_totaux = df_ca.groupby(['Residence', 'Annee'])[ca_cols].sum().reset_index()
    ca_totaux = ca_totaux.rename(
        columns={
            'Montant_R': 'CA_R_Total', 
            'Montant_P': 'CA_P_Total', 
            'Montant_B': 'CA_B_Total'
        }
    )

    # 2. Joindre les totaux au DataFrame des Charges/Marges
    df_ratios = pd.merge(df_pivot, ca_totaux, on=['Residence', 'Annee'], how='left')
    
    # 3. Calculer les Ratios (Poids de la ligne sur le CA total)
    for ctx in ['R', 'P', 'B']:
        montant_col = f'Montant_{ctx}'
        ca_col = f'CA_{ctx}_Total'
        ratio_col = f'Ratio_{ctx}'
        
        # Le ratio n'est pertinent que si le CA est non nul et la ligne est une charge/marge (pas le CA lui-même)
        df_ratios[ratio_col] = np.where(
            (df_ratios[ca_col] != 0) & (df_ratios['Groupe'] != "Chiffre d'affaire"), 
            df_ratios[montant_col] / df_ratios[ca_col], 
            0
        )
        
    # 4. Calculer la variation de ratio (R vs B) en points de pourcentage (pp)
    df_ratios['Delta_Ratio_RB_pp'] = (df_ratios['Ratio_R'] - df_ratios['Ratio_B']) * 100
    
    return df_ratios[df_ratios['Groupe'] != "Chiffre d'affaire"] # Exclure les lignes de CA pour l'analyse de ratio

df_ratios = calculer_ratios(df_pivot)

In [29]:
def create_ratio_chunk(row):
    # Contexte
    residence = row['Residence']
    annee = int(row['Annee'])
    ligne = row['Ligne_Analytique']
    groupe = row['Groupe']
    
    # Variables de ratio
    ratio_r_pct = f"{row['Ratio_R'] * 100:.2f}%"
    ratio_b_pct = f"{row['Ratio_B'] * 100:.2f}%"
    delta_pp = row['Delta_Ratio_RB_pp']
    
    # Interprétation
    if row['Groupe'] == 'Charge':
        performance = 'dégradation de l\'efficience' if delta_pp > 0 else 'amélioration de l\'efficience'
    else: # Marge
        performance = 'amélioration de la rentabilité' if delta_pp > 0 else 'dégradation de la rentabilité'
    
    # Chunk
    chunk = (
        f"ANALYSE DE RATIO ET D'EFFICIENCE | Résidence : {residence} | Année : {annee} | "
        f"Poste : {ligne} ({groupe}).\n\n"
        f"**POIDS SUR LE CA :**\n"
        f"   Ratio Réel (R) : {ratio_r_pct} | Ratio Budgété (B) : {ratio_b_pct}.\n"
        f"   Variation (R vs B) : **{delta_pp:+.2f} points de pourcentage (pp)**.\n"
        f"**CONCLUSION :** Cette variation représente une **{performance}** de la gestion de ce poste."
    )
    
    return chunk

# Génération des chunks de ratios
df_ratios['Chunk_Ratio'] = df_ratios.apply(create_ratio_chunk, axis=1)

# Ajout à la liste finale
new_chunks.extend(df_ratios['Chunk_Ratio'].tolist())
new_metadata.extend(df_ratios[['Residence', 'Annee', 'Code_H', 'Ligne_Analytique', 'Groupe', 'Ratio_R', 'Delta_Ratio_RB_pp']].to_dict('records'))

In [30]:
def create_factual_chunks(df_pivot):
    chunks = []
    
    for index, row in df_pivot.iterrows():
        residence = row['Residence']
        annee = int(row['Annee'])
        code_h = row['Code_H']
        ligne = row['Ligne_Analytique']
        groupe = row['Groupe']
        
        # 1. Chunk pour le montant Réel (R)
        chunk_r = (
            f"FAIT FINANCIER SIMPLE (RÉEL) | Résidence : {residence} | Année : {annee} | Poste : {ligne} ({code_h}).\n"
            f"Le montant **RÉEL** de ce poste pour l'exercice {annee} s'élève à **{row['Montant_R']:,.0f} €**."
        )
        chunks.append(chunk_r)
        
        # 2. Chunk pour le montant Budgété (B)
        chunk_b = (
            f"FAIT FINANCIER SIMPLE (BUDGÉTÉ) | Résidence : {residence} | Année : {annee} | Poste : {ligne} ({code_h}).\n"
            f"Le montant **BUDGÉTÉ** pour ce poste sur l'année {annee} était de **{row['Montant_B']:,.0f} €**."
        )
        chunks.append(chunk_b)
        
        # 3. Chunk pour le montant Prévisionnel (P)
        chunk_p = (
            f"FAIT FINANCIER SIMPLE (PRÉVISIONNEL) | Résidence : {residence} | Année : {annee} | Poste : {ligne} ({code_h}).\n"
            f"Le montant **PRÉVISIONNEL** (Forecast) de ce poste pour l'année {annee} était de **{row['Montant_P']:,.0f} €**."
        )
        chunks.append(chunk_p)
        
    return chunks

factual_chunks = create_factual_chunks(df_pivot)

factual_metadata = []
for index, row in df_pivot.iterrows():
    # R
    factual_metadata.append({
        'Residence': row['Residence'], 'Annee': int(row['Annee']), 
        'Ligne': row['Ligne_Analytique'], 'Contexte': 'R', 'Type_Chunk': 'Factuel Simple'
    })
    # B
    factual_metadata.append({
        'Residence': row['Residence'], 'Annee': int(row['Annee']), 
        'Ligne': row['Ligne_Analytique'], 'Contexte': 'B', 'Type_Chunk': 'Factuel Simple'
    })
    # P
    factual_metadata.append({
        'Residence': row['Residence'], 'Annee': int(row['Annee']), 
        'Ligne': row['Ligne_Analytique'], 'Contexte': 'P', 'Type_Chunk': 'Factuel Simple'
    })

new_chunks.extend(factual_chunks)
new_metadata.extend(factual_metadata)

In [31]:
def create_top_movers_chunks(df_pivot, top_n=3):
    chunks = []
    
    # Filtrer uniquement les lignes qui ont un impact sur l'écart global (charges et marges)
    df_impact = df_pivot[df_pivot['Groupe'].isin(['Charge', 'Marge'])].copy()
    
    for (residence, annee), group in df_impact.groupby(['Residence', 'Annee']):
        
        # 1. Identifier les impacts les plus DÉFAVORABLES
        # Pour les Charges (Ecart > 0 est défavorable), Marge (Ecart < 0 est défavorable)
        group['Impact_Defavorable'] = np.where(
            (group['Groupe'] == 'Charge') & (group['Ecart_R_B_Abs'] > 0), group['Ecart_R_B_Abs'], 
            np.where((group['Groupe'] == 'Marge') & (group['Ecart_R_B_Abs'] < 0), -group['Ecart_R_B_Abs'], 
            0)
        )
        top_defavorable = group.nlargest(top_n, 'Impact_Defavorable')
        
        # 2. Identifier les impacts les plus FAVORABLES
        group['Impact_Favorable'] = np.where(
            (group['Groupe'] == 'Charge') & (group['Ecart_R_B_Abs'] < 0), -group['Ecart_R_B_Abs'], 
            np.where((group['Groupe'] == 'Marge') & (group['Ecart_R_B_Abs'] > 0), group['Ecart_R_B_Abs'], 
            0)
        )
        top_favorable = group.nlargest(top_n, 'Impact_Favorable')
        
        # --- Création du chunk ---
        
        # Formatage des listes (Ligne, Écart Absolu)
        def format_top_list(df_top):
            return [f"{row['Ligne_Analytique']} ({row['Ecart_R_B_Abs']:,.0f} €)" for index, row in df_top.iterrows()]

        list_defavorable = format_top_list(top_defavorable)
        list_favorable = format_top_list(top_favorable)
        
        chunk = (
            f"SYNTHÈSE IMPACTS MAJEURS (TOP {top_n}) | Résidence : {residence} | Année : {annee} (R vs B).\n\n"
            f"**PRINCIPAUX ÉCARTS DÉFAVORABLES :**\n* " + "\n* ".join(list_defavorable) + "\n\n"
            f"**PRINCIPAUX ÉCARTS FAVORABLES :**\n* " + "\n* ".join(list_favorable)
        )
        
        chunks.append(chunk)
        new_metadata.append({'Residence': residence, 'Annee': annee, 'Analyse': f"Top {top_n} Ecarts R vs B"})

    return chunks

top_movers_chunks = create_top_movers_chunks(df_pivot, top_n=3)
new_chunks.extend(top_movers_chunks)

In [32]:
len(new_chunks), len(new_metadata)

(4135, 4135)

# Vectorisation et enregistrement dans chromadb

In [33]:
import chromadb
from sentence_transformers import SentenceTransformer

d:\SL_CB_IA\.env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [34]:
EMBEDDING_MODEL_NAME = 'distiluse-base-multilingual-cased-v2' 
embed_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

In [35]:
ids = [f"doc_{i}" for i in range(len(new_chunks))]

In [36]:
def clean_metadata_types(metadata_list):
    """
    Convertit récursivement les types NumPy (int64, float64, etc.) 
    en types Python natifs (int, float) dans une liste de dictionnaires.
    Ceci est essentiel pour la compatibilité avec ChromaDB.
    """
    cleaned_metadata = []
    for entry in metadata_list:
        cleaned_entry = {}
        for key, value in entry.items():
            # Conversion des types numériques NumPy
            if isinstance(value, (np.int64, np.int32, np.int8)):
                cleaned_entry[key] = int(value)
            elif isinstance(value, (np.float64, np.float32)):
                # Utiliser round() ou s'assurer que float(value) n'est pas NaN
                cleaned_entry[key] = float(value) if not np.isnan(value) else None
            elif isinstance(value, (int, float, str, bool, type(None))):
                cleaned_entry[key] = value
            else:
                # Gérer d'autres types non supportés si nécessaire (ex: convertir en str)
                cleaned_entry[key] = str(value)
        cleaned_metadata.append(cleaned_entry)
    return cleaned_metadata

new_metadata = clean_metadata_types(new_metadata)

In [37]:
def generate_embeddings(texts):
    """Génère les vecteurs à partir du texte."""
    # Note : Le calcul peut être long si le nombre de chunks est très élevé (milliers)
    return embed_model.encode(texts).tolist()

embeddings = generate_embeddings(new_chunks)

In [38]:
def indexer_chromadb(embeddings, chunks, metadata, ids):
    """Indexe les chunks dans ChromaDB."""
    try:
        # --- Configuration ChromaDB ---
        client = chromadb.Client()
        collection_name = 'residence_finance_analysis_complete'

        # Suppression pour un ré-indexage propre
        try:
            client.delete_collection(name=collection_name)
        except Exception:
            pass # Ignore si la collection n'existe pas

        collection = client.get_or_create_collection(
            name=collection_name, 
            metadata={"hnsw:space": "cosine"}
        )

        # Ajout des documents à la collection
        collection.add(
            embeddings=embeddings,
            documents=chunks,
            metadatas=metadata,
            ids=ids
        )

        print(f"\n✅ Indexation complète. {collection.count()} documents analytiques sont maintenant disponibles dans ChromaDB.")
        print("La base est prête pour l'interrogation par Ollama via RAG.")
    except Exception as e:
        print(f"Erreur lors de l'indexation: {e}")

# Lancement de l'indexation
indexer_chromadb(embeddings, new_chunks, new_metadata, ids)


✅ Indexation complète. 4135 documents analytiques sont maintenant disponibles dans ChromaDB.
La base est prête pour l'interrogation par Ollama via RAG.


# Asking Ollama

In [39]:
from langchain_ollama  import OllamaLLM
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [40]:
# --- Configuration Globale ---
# Nom de la collection ChromaDB précédemment créée
COLLECTION_NAME = 'residence_finance_analysis_complete'
# Modèle d'embeddings utilisé pour l'indexation (DOIT être le même que celui d'indexation)
EMBEDDING_MODEL_NAME = 'distiluse-base-multilingual-cased-v2' 
# Modèle Ollama à utiliser (doit être téléchargé et lancé via 'ollama run <modele>')
OLLAMA_MODEL = "llama3:8b" #T config.GP Choisissez votre modèle Ollama (ex: llama3, gpt-oss)
OLLAMA_BASE_URL = "http://si-5/" # URL par défaut d'Ollama
# Seuil de récupération (Top K)
K_RETRIEVAL = 5

all-mpnet-base-v2

distiluse-base-multilingual-cased-v2

In [41]:
# 1. Initialisation du modèle d'embeddings pour le Retrieval
embed_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

In [42]:
class SentenceTransformerEmbeddings:
    """Wrapper simple pour LangChain sur SentenceTransformer."""
    def embed_documents(self, texts):
        return embed_model.encode(texts).tolist()
    def embed_query(self, text):
        return embed_model.encode(text).tolist()

# 2. Chargement de la Base Vectorielle (ChromaDB)
vectorstore = Chroma(
    collection_name=COLLECTION_NAME,
    embedding_function=SentenceTransformerEmbeddings(),
)

# 3. Création du Retriever
# Le retriever est l'outil qui recherche les chunks les plus pertinents (Top K)
retriever = vectorstore.as_retriever(search_kwargs={"k": K_RETRIEVAL})
print(f"✅ Retriever ChromaDB initialisé pour récupérer les Top {K_RETRIEVAL} chunks.")

✅ Retriever ChromaDB initialisé pour récupérer les Top 5 chunks.


In [43]:
# 1. Connexion au modèle Ollama
llm = OllamaLLM(model=OLLAMA_MODEL, base_url=OLLAMA_BASE_URL)
print(f"✅ Connexion à Ollama ({OLLAMA_MODEL}) établie.")

✅ Connexion à Ollama (llama3:8b) établie.


In [ ]:
# 1. Template de Prompt (Instructions pour Ollama)
template = """
Tu es un Contrôleur de Gestion français expert et rigoureux pour des résidences.
Ta mission est d'analyser les données budgétaires et les écarts fournis ci-dessous dans le 'CONTEXTE FACTUEL'.
Réponds à la question de manière factuelle, précise et professionnelle.
Si les informations demandées ne sont pas dans le contexte, indique poliment que tu ne peux pas répondre.

CONTEXTE FACTUEL:
{context}

QUESTION:
{question}
"""
prompt = ChatPromptTemplate.from_template(template)

# 2. Construction de la Chaîne RAG (LangChain Expression Language - LCEL)
# La chaîne suit le flux : Question -> Récupération des chunks -> Formatage du Prompt -> Appel à Ollama

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} 
    | prompt 
    | llm
    | StrOutputParser() # Convertit la sortie du LLM en simple chaîne de caractères
)

In [45]:
# --- Exemple de Requête ---
query_1 = "Pour la résidence 11000099 - Noisy en 2024, quel est le poste de charge qui a le plus dégradé le Taux de Marge par rapport au Budget, et quelle était l'évolution de ce poste par rapport à 2023 ?"

# Exécution de la chaîne
try:
    response = rag_chain.invoke(query_1)
    
    print("\n" + "="*50)
    print(f"QUESTION: {query_1}")
    print("="*50)
    print(f"RÉPONSE D'OLLAMA:\n{response}")
    print("="*50)

except Exception as e:
    print(f"\nUne erreur est survenue lors de l'appel RAG : {e}")
    print("Vérifiez que le serveur Ollama est démarré et que le modèle spécifié est chargé.")


QUESTION: Pour la résidence 11000099 - Noisy en 2024, quel est le poste de charge qui a le plus dégradé le Taux de Marge par rapport au Budget, et quelle était l'évolution de ce poste par rapport à 2023 ?
RÉPONSE D'OLLAMA:
Je suis désolé, mais je ne peux pas répondre à cette question car les informations demandées ne sont pas fournies dans le contexte factuel fourni. En effet, il n'y a pas de données pour l'année 2024.

Cependant, si vous avez des questions sur les écart entre les postes de charge ou sur l'évolution de certains postes de charge par rapport à une année précédente, je serais ravi de vous aider en utilisant les informations fournies dans le contexte factuel.


In [46]:
# --- Exemple de Requête 2 (Analyse de Tendance) ---
query_2 = "Décris l'évolution de la rentabilité (Marge) de la résidence 11000099 entre 2022 et 2024."

try:
    response_2 = rag_chain.invoke(query_2)
    
    print("\n" + "="*50)
    print(f"QUESTION: {query_2}")
    print("="*50)
    print(f"RÉPONSE D'OLLAMA (Tendance):\n{response_2}")
    print("="*50)
except Exception as e:
    pass


QUESTION: Décris l'évolution de la rentabilité (Marge) de la résidence 11000099 entre 2022 et 2024.
RÉPONSE D'OLLAMA (Tendance):
As a Contrôleur de Gestion expert, I will analyze the provided data to answer your question.

Firstly, I would like to highlight that there is no information available for 2023. Therefore, it is not possible to provide an analysis of the evolution of the profitability (Marge) of the residence 11000099 between 2022 and 2024.

However, I can analyze the provided data for 2022 and 2025 to get some insights on the trends.

For 2022:

* The budgeted amount for "Taxe foncière / taxe d'habitation" is 0 €.
* There is no information available on other income or expenses that would impact the profitability of the residence.

For 2025:

* The budgeted amount for "Taxe foncière / taxe d'habitation" is 1,234 €.
* The budgeted amount for "Impayés locataires" is -10,810 €.
* There is no information available on other income or expenses that would impact the profitability o

In [47]:
# --- Exemple de Requête ---
query = "Quelle est la recette réelle de noisy en 2024 ?"

# Exécution de la chaîne
try:
    response = rag_chain.invoke(query)
    
    print("\n" + "="*50)
    print(f"QUESTION: {query}")
    print("="*50)
    print(f"RÉPONSE D'OLLAMA:\n{response}")
    print("="*50)

except Exception as e:
    print(f"\nUne erreur est survenue lors de l'appel RAG : {e}")
    print("Vérifiez que le serveur Ollama est démarré et que le modèle spécifié est chargé.")


QUESTION: Quelle est la recette réelle de noisy en 2024 ?
RÉPONSE D'OLLAMA:
Je suis désolé mais je ne dispose pas des informations nécessaires pour répondre à cette question. Le contexte fourni ne contient pas de données sur l'année 2024, et notamment la recette réelle de Noisy en 2024. Pour fournir une réponse précise et factuelle, j'ai besoin d'informations supplémentaires qui ne sont pas disponibles dans le contexte actuel.


In [48]:
# --- Exemple de Requête ---
query = "Analyse la résidence de noisy."

# Exécution de la chaîne
try:
    response = rag_chain.invoke(query)
    
    print("\n" + "="*50)
    print(f"QUESTION: {query}")
    print("="*50)
    print(f"RÉPONSE D'OLLAMA:\n{response}")
    print("="*50)

except Exception as e:
    print(f"\nUne erreur est survenue lors de l'appel RAG : {e}")
    print("Vérifiez que le serveur Ollama est démarré et que le modèle spécifié est chargé.")


QUESTION: Analyse la résidence de noisy.
RÉPONSE D'OLLAMA:
En analysant les données budgétaires fournies, voici quelques éléments clés que je retiens à propos de la résidence de Noisy :

* Dans le domaine du chiffre d'affaire (poste 1.1), il est notable que l'année 2024 a enregistré une augmentation de 6,15% par rapport à l'année 2023, ce qui constitue un résultat positif pour la résidence.
* Cependant, dans le même domaine (poste 1.1), l'année 2025 présente une diminution de 100% par rapport à l'année 2024, ce qui est un résultat négatif.

Dans le domaine des charges (poste 2.x), il est notable que la récupération sur les locataires a enregistré une diminution de 68,06% entre 2022 et 2023, mais cette tendance est considérée comme positive sur la marge pour la résidence.

En résumé, la résidence de Noisy présente un contexte financier complexe, avec des éléments positifs (augmentation du chiffre d'affaire en 2024) et des éléments négatifs (diminution du chiffre d'affaire en 2025). Il 

In [49]:
# --- Exemple de Requête ---
query = "Quelle est le chiffre d'affaire réel de noisy en 2024 ?"

# Exécution de la chaîne
try:
    response = rag_chain.invoke(query)
    
    print("\n" + "="*50)
    print(f"QUESTION: {query}")
    print("="*50)
    print(f"RÉPONSE D'OLLAMA:\n{response}")
    print("="*50)

except Exception as e:
    print(f"\nUne erreur est survenue lors de l'appel RAG : {e}")
    print("Vérifiez que le serveur Ollama est démarré et que le modèle spécifié est chargé.")


QUESTION: Quelle est le chiffre d'affaire réel de noisy en 2024 ?
RÉPONSE D'OLLAMA:
Je suis désolé, mais comme Contrôleur de Gestion expert et rigoureux, je ne peux pas répondre à cette question car les informations demandées ne sont pas fournies dans le contexte factuel. Aucun chiffre d'affaire réel pour l'année 2024 n'est mentionné dans ces documents.

Je suis prêt à analyser les données budgétaires et les écarts si vous avez des informations supplémentaires ou spécifiques sur laquelle je peux me concentrer.


In [50]:
text = """
**Analyse synthétique de la résidence Noisy (Résidence 11000099 – Noisy)**  

| Période | Indicateur | Valeur réelle | Valeur budgétaire | Écart | Signification |
|---------|------------|---------------|-------------------|-------|---------------|
| **2023 → 2024** | **Loyers logements & parkings HT** | 2 026 364 € | – | + 117 454 € (6,15 %) | Croissance positive du chiffre d’affaires, bénéfique pour la trésorerie et la capacité d’investissement. |
| **2023** | **Récupération sur les locataires** (Charge) | -0,34 % du CA | 0,00 % | -0,34 % | Amélioration de l’efficacité : la charge est inférieure à la prévision, ce qui libère de la marge. |
| **2024** | **Taxe foncière / Taxe d’habitation** (Charge) | 0,02 % du CA | 0,00 % | +0,02 % | Dégradation de l’efficience : la charge dépasse la prévision, compressant la marge. |
| **2026** | **Taxe foncière / Taxe d’habitation** (Charge) | 0,00 % du CA | 0,04 % / 0,02 % | –0,04 % / –0,02 % | Amélioration attendue : la charge se situe en dessous du budget, indiquant une meilleure maîtrise des coûts. |

---

### 1.  Revenus

- **Croissance YoY de 6,15 %** sur le poste *Loyers logements et parkings HT* (2023 → 2024).  
  - La hausse de **117 454 €** confirme une demande locative stable et permet d’augmenter la capacité de couverture des charges fixes.  
  - Cette évolution positive est un point fort du plan financier pour l’année en cours.

### 2.  Charges opérationnelles

| Poste | 2023 | 2024 | 2026 | Observation |
|-------|------|------|------|-------------|
| *Récupération sur les locataires* | -0,34 % vs 0,00 % | – | – | Charge maîtrisée, marge opérationnelle plus élevée. |
| *Taxe foncière / Taxe d’habitation* | – | +0,02 % vs 0,00 % | 0,00 % vs 0,04 % / 0,02 % | 2024 : dépassement du budget → dégradation de l’efficience. 2026 : baisse attendue, amélioration possible grâce à une meilleure négociation ou un changement de régime fiscal. |

- **Efficience** : La résidence a réalisé des gains d’efficacité sur le poste de récupération locataire, mais a rencontré une dégradation sur les taxes foncières en 2024.  
- **Perspectives 2026** : Les écarts négatifs (‑0,04 % et ‑0,02 %) indiquent une amélioration par rapport aux prévisions budgétaires, ce qui devrait rétablir l’équilibre sur ce poste.

### 3.  Points d’attention & recommandations

1. **Surveillance continue des taxes foncières**  
   - Vérifier les raisons de la hausse en 2024 (augmentation du taux, changement de valeur cadastrale).  
   - Explorer les options de subvention ou de réévaluation pour contenir le coût.

2. **Optimisation de la récupération locataire**  
   - Continuer à maintenir la discipline déjà obtenue pour consolider la marge.  
   - Mettre à jour les procédures de relance afin d’éviter tout risque de dégradation future.

3. **Capitalisation sur la croissance des loyers**  
   - Utiliser la hausse de 6,15 % pour renforcer la trésorerie ou financer des travaux d’amélioration du parc (énergie, accessibilité).  
   - Revoir la politique de location afin d’étendre l’incrément de revenu si le marché le permet.

4. **Prévision budgétaire ajustée**  
   - Intégrer les écarts constatés (positifs et négatifs) pour réviser les prévisions 2025‑2027, afin de rendre les objectifs plus réalistes et alignés sur la performance réelle.

---

### Conclusion

La résidence Noisy présente un **tendance positive** sur le chiffre d’affaires locatif (croissance de 6,15 % entre 2023 et 2024) et a réussi à **améliorer l’efficacité** sur le poste de récupération locataire. Cependant, la hausse des taxes foncières en 2024 a temporairement **dégradé l’efficience** de la résidence. Les prévisions pour 2026 montrent un **rétablissement** attendu de la charge fiscale par rapport au budget, ce qui est rassurant.

En résumé, la résidence Noisy affiche de bons résultats sur le volet revenus et contrôle des charges, mais nécessite un suivi ciblé des taxes foncières pour éviter toute régression future.
"""

In [51]:
text2 = """
**Analyse de la résidence Noisy (code 11000099)**  

| Période | Indicateur | Valeur réelle | Variation YoY | Impact déclaré |
|---------|------------|---------------|---------------|----------------|
| 2022‑2023 | Loyers logements & parkings HT | 1 908 910 € | +11,26 % ( +193 166 € ) | Positif |
| 2023‑2024 | Loyers logements & parkings HT | 2 026 364 € | +6,15 % ( +117 454 € ) | Positif |
| 2024‑2025 | Loyers logements & parkings HT | 0 € | –100 % ( –2 026 364 € ) | Négatif |
| 2022‑2023 | Vaisselle logement & cafétéria | 3 634 € | –23,33 % ( –1 106 € ) | Positif sur la marge |
| 2022‑2023 | Récupération sur les locataires | –21 160 € | –68,06 % ( –8 569 € ) | Positif sur la marge |

### 1. **Chiffre d’affaires (loyers & parkings)**
- **Croissance 2023/2022** de 11,26 % (193 k €) et **2024/2023** de 6,15 % (117 k €).  
  La tendance est globalement positive et conforme aux objectifs de la résidence pour les deux premières années consécutives.  
- **Anomalie 2025** : le chiffre d’affaires passe à 0 € – perte de 2 026 364 € (-100 %).  Cette chute massive indique un problème majeur (fermeture temporaire, interruption de service, perte de contrats majeurs…) qui nécessite une investigation immédiate.

### 2. **Charges opérationnelles**
- **Vaisselle logement & cafétéria** : baisse de 23,33 % (‑1 106 €).  Cela améliore la marge brute et est considéré comme un effet positif.
- **Récupération sur les locataires** : la valeur devient plus négative (‑21 160 € vs ‑12 591 €).  Bien que le montant global diminue, l’analyse de la marge indique que cette charge est compensée par des gains ou des économies ailleurs, d’où son impact « positif sur la marge ».

### 3. **Synthèse de la performance**
- **2023‑2024** : performance satisfaisante – croissance du chiffre d’affaires et réduction des charges, ce qui renforce la marge nette.
- **2025** : la disparition du chiffre d’affaires est un signal d’alarme.  Sans explication supplémentaire (non fournie dans les documents), elle compromet la viabilité financière de la résidence pour cette année et peut entraîner un redressement budgétaire ou un ajustement des prévisions.

### 4. **Recommandations**
1. **Enquête de cause** sur la chute de 100 % du chiffre d’affaires en 2025 (contrats, incidents opérationnels, réglementaires, etc.).  
2. **Mise en place de mesures de redressement** si la perte est liée à des actions correctives (révision des tarifs, réactivation de logements, renegociation de contrats).  
3. **Suivi rapproché** des charges (vaisselle, récupération) afin de maintenir la dynamique de réduction des coûts observée entre 2022 et 2023.  

En résumé, la résidence Noisy a affiché une croissance saine et une amélioration de la marge jusqu’en 2024, mais la situation budgétaire en 2025 doit être immédiatement clarifiée pour éviter un impact financier négatif majeur.
"""

In [52]:
html_text = format_response(text)
display(HTML(html_text))

In [53]:
html_text2 = format_response(text2)
display(HTML(html_text2))

Période,Indicateur,Valeur réelle,Variation YoY,Impact déclaré
2022‑2023,Loyers logements & parkings HT,1 908 910 €,"+11,26 % ( +193 166 € )",Positif
2023‑2024,Loyers logements & parkings HT,2 026 364 €,"+6,15 % ( +117 454 € )",Positif
2024‑2025,Loyers logements & parkings HT,0 €,–100 % ( –2 026 364 € ),Négatif
2022‑2023,Vaisselle logement & cafétéria,3 634 €,"–23,33 % ( –1 106 € )",Positif sur la marge
2022‑2023,Récupération sur les locataires,–21 160 €,"–68,06 % ( –8 569 € )",Positif sur la marge
